In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [19]:
#needed packages to install
#!pip install texthero==1.0.5
# !pip install gensim
# !pip install openpyxl 
# !pip install bnlp_toolkit
# !pip install python-bidi
# !pip install texthero
# !pip install bangla-stemmer

     |████████████████████████████████| 1.5 MB 9.2 MB/s 
     |████████████████████████████████| 24.2 MB 1.4 MB/s 
     |████████████████████████████████| 235 kB 55.5 MB/s 
     |████████████████████████████████| 749 kB 48.7 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
  Attempting uninstall: gensim
    Found existing installation: gensim 4.0.1
    Uninstalling gensim-4.0.1:
      Successfully uninstalled gensim-4.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bnlp-toolkit 3.1.2 requires gensim==4.0.1, but you have gensim 3.8.3 which is incompatible.


In [20]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from wordcloud import WordCloud
from bnlp.corpus import stopwords, punctuations
import bnlp
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
from bnlp import BasicTokenizer,NLTKTokenizer
from bangla_stemmer.stemmer.stemmer import BanglaStemmer
import warnings
warnings.filterwarnings("ignore")


In [22]:
df = pd.read_excel('/content/AugExtented_Final_Combained_Data_Final.xlsx')


df_plot=df.copy()
df.head()

Speech  Label
0  ওনাদের খাবারের পরিমাণ দেখে আমরা পুরাই অবাক, 55...      0
1  করুণ খাবারের মান। বেশি দাম. এটা এতটাই খারাপ যে...      0
2    বাজে ছিল , খাবার গরম করে দিলে হয়তবা ভাল লাগত ।      0
3  তবে সমস্যা যেটা হয়েছে, স্টেক এতই হার্ড ছিলো যে...      0
4                     তেহেরি খুব একটা ভালো লাগে নাই।      0

In [23]:
from sklearn.utils import shuffle
df = shuffle(df)
df=df.reset_index(drop=True)
df.head()

Speech  Label
0                            সমুচা আরেকটু বেশি দরকার      2
1  বাসাবোর কাবাবিস,কাবাব মে হাড্ডি, কাবাব স্টেশন ...      1
2                                       রোস্ট এভারেজ      2
3               মাছের তরকারি আর আরেক্টু বাড়ালে জমতো      2
4                               রোস্ট আহামরি কিছু না      2

In [24]:
df_copy=df.copy()

In [25]:
def demoji(text):
	emoji_pattern = re.compile("["
		u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00010000-\U0010ffff"
	                           "]+", flags=re.UNICODE)
	return(emoji_pattern.sub(r'', text)) 


def clean(text):
    text = re.sub('[%s]' % re.escape(punctuations), ' ', text)     #escape punctuation
    text = re.sub('\n', ' ', text)                                 #replace line break with space
    text = re.sub('\w*\d\w*', ' ', text)                           #ignore digits
    #text = re.sub('\xa0', ' ', text)                              
    return text

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[\u09E6-\u09FF]+', ' ', text)                  #remove bangla punctuations
    return text

In [26]:
df[u'text'] = df[u'Speech'].astype(str)
df[u'text'] = df[u'text'].apply(lambda x:demoji(x))
df['text'] = df['text'].apply(lambda x: re.split('http:\/\/.*', str(x))) #remove urls
df["text"] = df['text'].apply(lambda x: clean(str(x)))                      
df['text'] = df['text'].apply(lambda x: remove_punct(x))

In [27]:
#remove special characters
spec_chars = ["!",'"',"।","#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df['text'] = df['text'].str.replace(char, ' ') 
    df['text'] = df['text'].str.split().str.join(' ')         #remove whitespace

In [28]:
custom_stop_word_list=['আমার ','অথচ ','অথবা ','অনুযায়ী ','অনেক ','অনেকে ','অনেকেই ','অন্তত ','অন্য ','অবধি ','অবশ্য ','অর্থাত ','আই ','আগামী ','আগে ','আগেই ','আছে ','আজ ','আদ্যভাগে ',
                       'আপনার ','আপনারা ','আপনি ','আবার ','আসবে ','আমরা ',' আমাকে ','আমাদের ','আমার ','আমি ','আর ','আরও ','ইত্যাদি ','ইহা ','উচিত ','উত্তর ','উনি ','উপর ','উপরে ','এ ','এঁদের ','এঁরা ','এরা ',
                       'এই ','একই ','একটি ','একবার ','একে ','এক্ ','এখন ','এখনও ','এখানে ','এখানেই ','এটা ','এটাই ','এটি ','এত ','এতটাই ','এতে ','এদের ','এব ','এবং ','এবার ','এমন ','এমনকী ',
                       'এমনি ','এর ','এরা ','এল ','এস ','এসে ','ঐ ','ওঁদের ','ওঁর ','ওঁরা ','ওই ','ওকে ','ওখানে ','ওদের ','ওর ','ওরা ','কখনও ','কত ','কবে ','কমনে ','কয়েক ','কয়েকটি ','করছে ',
                       'করছেন ','করতে ',' করবে',' করবেন',' করলে ',' করলেন',' করা',' করাই',' করায়',' করার',' করি','করতে ','করিতে ','করিয়া ','করিয়ে ','করে ','করেই ','করেছিলেন ','করেছে ','করেছেন ','করেন ',
                       'কাউকে ','কাছ ','কাছে ','কাজ ','কাজে ','কারও ','কারণ ','কি ','কিংবা ','কিছু ','কিছুই ','হেতি ','কিন্তু ','ন্তু ','কী ','কে ','কেউ ','কেউই ','কেখা ','কেন ','কোটি ','কোন ','কোনও ',
                       'কোনো ','ক্ষেত্রে ','কয়েক ','খুব ','গিয়ে ','গিয়েছে ','গেছেন ','গিয়ে ','গুলি ','গেছে ','গেল ','গেলে ','গোটা ','চলে ','চান ','চায় ','চার ','চালু ','চেয়ে ','চেষ্টা ','ছাড়া ','ছাড়াও ','ছিল ','ছিলেন ','জন ',
                       'জনকে ','জনের ','জন্য ','জন্যওজে ','জানতে ','জানা ','জানানো ','জানায় ','জানিয়ে ','জানিয়েছে ','জ্নজন ','জন ','টা ','টি ','ঠিক ','তখন ','তত ','তথা ','তবু ','তবে ','তা ','তাঁকে ','তাঁদের ',
                       'তাঁর ','তোর ','তাঁরা ','তাঁহারা ','তাই ','যে ''তাও ','তাকে ','তাতে ','তাদের ','তার ','তারপর ','তারা ','তারৈ ','তাহলে ','তাহা ','তাহাতে ' ,'তাহার ','তিনঐ ','তিনি ','তিনিও ','তুমি ','তুলে ','তেমন ','তো ','তোমার ',
                       'থাকবে ','থাকবেন ','থাকা ','থাকায় ','থাকে ','থাকেন ','থেকে ','থেকেই ','থেকেও ','দিকে ','দিতে ','দিতাম','দিন ','দিয়ে ','দিয়েছে ','দিয়েছেন ','দিলেন ', 'দু ','দুই ','দুটি ','দুটো ','দেওয়া ','দেওয়ার ','দেওয়া ',
                       'দেখতে ','দেখা ','দেখে ','দেন ','দেয়া ','দেয় ','দ্বারা ','ধরা ','ধরে ','ধামার ','নতুন ','নাই ','নাকি ','নাগাদ ','নানা ','নিজে ','নিজেই ','নিজেদের ','নিজের ','নিতে ','নিয়ে ','নিয়ে ','নেই ','নেওয়া ','নেওয়ার ',
                       'নেওয়া ','নয় ','পক্ষে ','পর ','পরে ','পরেই ','পরেও ','পর্যন্ত ','পাওয়া ','পাচ ','পারি ','পারে ','পারেন ','পেয়ে ','পেয়্র্ ','প্রতি ','প্রথম ','প্রভৃতি ','প্রযন্ত ','প্রাথমিক ','প্রায় ','প্রায় ','ফলে ','ফিরে ','ফের ',
                       'বক্তব্য ','বদলে ','বন ','বরং ','বলতে ','বলছি ','বলল ','বললেন ','বলা ','বলে ','বলেছেন ','বলেন ','বসে ','বহু' ,'বাদে ','বার ','বিনা ','বিভিন্ন ','বিশেষ ','বিষয়টি ','বেশ ','বেশি ','ব্যবহার ','ব্যাপারে ','ভাবে ', 'ভাবেই ',
                       'মতো ','মতোই ','মধ্যভাগে ','মধ্যে ','মধ্যেই ','মধ্যেও ','মনে ','মাত্র ','মাধ্যমে ','মোট ','মোটেই ','যখন ','যত ','যতটা ','যথেষ্ট ','যদি ','যদিও ','যা ','যাঁর ','যাঁরা ','যাওয়া ','যাওয়ার ','যাওয়া ','যাকে ','যাচ্ছে ',
                       'যাতে ','যাদের ','যান ','যাবে ','যায় ','যার ','যারা ','যিনি ','অতএব ','যেখানে ','যেতে ','যেন ','যেমন ','রকম ','রয়েছে ','রাখা ','রেখে ','লক্ষ ','শুধু ','শুরু ','সঙ্গে ','সঙ্গেও ','সব ','সবার ','সবাইর ','সমস্ত ',
                       'সম্প্রতি ','সহ ','সহিত ','সবই ','সাধারণ ','সামনে ','সুতরাং ','সবাইর ','সে ','সেই ','সেখান ','সেখানে ','সেটা ', 'সেটাই ','সেটাও ','সেটি ','স্পষ্ট ','স্বয়ং ','হইতে ','হইবে ','হইয়া ','হওয়া ','হওয়ায় ','হওয়ার ','হচ্ছে ','হত ','হতে ',
                       'লেগেছে ','হতেই ','হন ','হইত ','হবে ','তিনি ','হবেন ','হয় ','হয়তো ','হয়নি ','হয়ে ','হয়েই ','হয়েছিল ','হয়েছে ','হয়েছেন ','হল ','হলে ','হলেই ','হলেও ','হলো ','হাজার ','হিসাবে ','হৈলে ','হোক ','হয় ']

#digits=['০ ','১ ','২ ','৩ ','৪ ','৫ ','৬ ','৭ ','৮ ','৯ ']

final_stopword_list = custom_stop_word_list 

pat = r'\b(?:{})\b'.format('|'.join(final_stopword_list))
df['text'] = df['text'].str.replace(pat, ' ')
df['text'] = df['text'].str.replace(r'\s+', ' ')

In [29]:
df["clean_text"]=df["text"]

In [30]:
def stemming_text(corpus):
    stm = BanglaStemmer()
    return [' '.join([stm.stem(word) for word in review.split()]) for review in corpus]

df['text'] = stemming_text(df['text'])


Streaming output truncated to the last 5000 lines.
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules

In [31]:
df.head()

Speech  Label  \
0                            সমুচা আরেকটু বেশি দরকার      2   
1  বাসাবোর কাবাবিস,কাবাব মে হাড্ডি, কাবাব স্টেশন ...      1   
2                                       রোস্ট এভারেজ      2   
3               মাছের তরকারি আর আরেক্টু বাড়ালে জমতো      2   
4                               রোস্ট আহামরি কিছু না      2   

                                              text  \
0                                 সমুচা আরেকটু দরক   
1  বাসাবোর কাবাব কাবাব মে হাড্ডি কাবাব স্টেশন ভালো   
2                                     রোস্ট এভারেজ   
3                    মাছ তরকারি আরেক্টু বাড়ালে জম   
4                                  রোস্ট আহামরি না   

                                          clean_text  
0                                 সমুচা আরেকটু দরকার  
1  বাসাবোর কাবাবিস কাবাব মে হাড্ডি কাবাব স্টেশন ভালো  
2                                       রোস্ট এভারেজ  
3                  মাছের তরকারি আরেক্টু বাড়ালে জমতো  
4                                    রোস্ট আহামরি না

In [32]:
#Tokenization

b_token = BasicTokenizer()
df['tokenized_clean_text'] = df.apply(lambda row: b_token.tokenize(row['clean_text']), axis=1)
df['tokenized_stem_text'] = df.apply(lambda row: b_token.tokenize(row['text']), axis=1)

df['token_length'] = df.apply(lambda row: len(row['tokenized_clean_text']), axis=1)

df.head() 

Speech  Label  \
0                            সমুচা আরেকটু বেশি দরকার      2   
1  বাসাবোর কাবাবিস,কাবাব মে হাড্ডি, কাবাব স্টেশন ...      1   
2                                       রোস্ট এভারেজ      2   
3               মাছের তরকারি আর আরেক্টু বাড়ালে জমতো      2   
4                               রোস্ট আহামরি কিছু না      2   

                                              text  \
0                                 সমুচা আরেকটু দরক   
1  বাসাবোর কাবাব কাবাব মে হাড্ডি কাবাব স্টেশন ভালো   
2                                     রোস্ট এভারেজ   
3                    মাছ তরকারি আরেক্টু বাড়ালে জম   
4                                  রোস্ট আহামরি না   

                                          clean_text  \
0                                 সমুচা আরেকটু দরকার   
1  বাসাবোর কাবাবিস কাবাব মে হাড্ডি কাবাব স্টেশন ভালো   
2                                       রোস্ট এভারেজ   
3                  মাছের তরকারি আরেক্টু বাড়ালে জমতো   
4                                    রোস্ট আহামরি না   

                                tokenized_clean_text  \
0                             [সমুচা, আরেকটু, দরকার]   
1  [বাসাবোর, কাবাবিস, কাবাব, মে, হাড্ডি, কাবাব, স...   
2                                    [রোস্ট, এভারেজ]   
3            [মাছের, তরকারি, আরেক্টু, বাড়ালে, জমতো]   
4                                [রোস্ট, আহামরি, না]   

                                 tokenized_stem_text  token_length  
0                               [সমুচা, আরেকটু, দরক]             3  
1  [বাসাবোর, কাবাব, কাবাব, মে, হাড্ডি, কাবাব, স্ট...             8  
2                                    [রোস্ট, এভারেজ]             2  
3                [মাছ, তরকারি, আরেক্টু, বাড়ালে, জম]             5  
4                                [রোস্ট, আহামরি, না]             3

In [33]:
df.describe()

Label  token_length
count  3963.000000   3963.000000
mean      1.163260      6.069644
std       0.833028      6.858650
min       0.000000      0.000000
25%       0.000000      3.000000
50%       1.000000      4.000000
75%       2.000000      6.000000
max       2.000000    103.000000

In [34]:
df1=df.copy()


df.to_csv(encoding='utf-8', index=False)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/augmented_tokenized.csv', index = False, header=True)
df.tail()

Speech  Label                        text  \
3958  খাবারের মান ভালো তবে পরিমাণ কম      2  খাবারের মান ভালো পরিমাণ কম   
3959              রোস্ট মোটামটি ভালো      2            রোস্ট মোটাম ভালো   
3960  কাচ্চি আরও রান্না হলে ভালো হতো      2       কাচ্চি রান্না ভালো হত   
3961                 সমুচা খুবই বাজে      0              সমুচা খুব বাজে   
3962         সবজি টেস্ট খুবই অসাধারণ      1      সবজি টেস্ট খুব অসাধারণ   

                      clean_text              tokenized_clean_text  \
3958  খাবারের মান ভালো পরিমাণ কম  [খাবারের, মান, ভালো, পরিমাণ, কম]   
3959          রোস্ট মোটামটি ভালো            [রোস্ট, মোটামটি, ভালো]   
3960      কাচ্চি রান্না ভালো হতো       [কাচ্চি, রান্না, ভালো, হতো]   
3961             সমুচা খুবই বাজে               [সমুচা, খুবই, বাজে]   
3962     সবজি টেস্ট খুবই অসাধারণ      [সবজি, টেস্ট, খুবই, অসাধারণ]   

                   tokenized_stem_text  token_length  
3958  [খাবারের, মান, ভালো, পরিমাণ, কম]             5  
3959              [রোস্ট, মোটাম, ভালো]             3  
3960        [কাচ্চি, রান্না, ভালো, হত]             4  
3961                [সমুচা, খুব, বাজে]             3  
3962       [সবজি, টেস্ট, খুব, অসাধারণ]             4

In [41]:
from bnlp import POS
df.tokenized_clean_text = df.tokenized_clean_text.astype(str)
def pos_tagging(doc):
  bn_pos = POS()
  model_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/bn_pos.pkl"
  doc = bn_pos.tag(model_path,doc)
  return doc

In [42]:
def get_postags(row):
    postags = pos_tagging(row["tokenized_clean_text"])
    list_classes = list()
    for  word in postags:
        list_classes.append(word[1])
    return list_classes

df["postags_list"] = df.apply(lambda row: get_postags(row), axis = 1)
df['postags_list'] = df.postags_list.apply(lambda x: [i for i in x if i != 'PU'])
df['postags_list'] = df.postags_list.apply(lambda x: [i for i in x if i != 'RDS'])
df.sample(10, random_state = 4)

Speech  Label  \
1780                             খুব একটা ভালো না খাবার      0   
3102                  বার্গার ভালো ছিল কিন্তু দামে বেশি      2   
1035                               পোলাও মুটামুটি ভালো       2   
693                              মাংস আরেকটু বেশি দরকার      2   
3887  বিরিয়ানির স্বাদ মোটামুটি ছিল। এবং কাবাব পুরান...      2   
3753  খাবার ভালো, তবে বাদাম ভর্তা দুইটা ও শুটকি ভর্ত...      1   
1560                                       রোস্ট এভারেজ      2   
90    মশলার সমৃদ্ধি এবং খুব হালকা মিষ্টি মিষ্টি অনুভ...      2   
3261  ফুচকা সবগুলো ভাঙা ছিলো একদম। খুবই বাজে এক্সপের...      0   
1472                  ভাল ছিল কিন্তু ঠান্ডা খাবার পাইছি      1   

                                                   text  \
1780                                    এক ভালো না খাবা   
3102                             বার্গার ভালো দামে বেশি   
1035                                   পোলা মুটামু ভালো   
693                                     মাংস আরেকটু দরক   
3887  বিরিয়ানির স্বাদ মোটামু কাবাব পুরানো খেত পছন্দ নি   
3753         খা ভালো বাদাম ভর্ দুই ও শুটকি ভর্ এক চাইছি   
1560                                       রোস্ট এভারেজ   
90    মশল সমৃদ্ধি হালকা মিষ্ মিষ্ অনুভূতি স্বাদ মোটা...   
3261       ফুচকা সব ভাঙা ছিলো একদম খুব বাজে এক্সপেরিন্স   
1472                                  ভাল ঠান্ডা খা পাই   

                                             clean_text  \
1780                                 একটা ভালো না খাবার   
3102                             বার্গার ভালো দামে বেশি   
1035                                  পোলাও মুটামু ভালো   
693                                   মাংস আরেকটু দরকার   
3887  বিরিয়ানির স্বাদ মোটামু কাবাব পুরানো খেতে পছন্...   
3753  খা ভালো বাদাম ভর্ দুইটা ও শুটকি ভর্ একটা চাইছিলাম   
1560                                       রোস্ট এভারেজ   
90    মশলার সমৃদ্ধি হালকা মিষ্ মিষ্ অনুভূতি স্বাদও ম...   
3261  ফুচকা সবগুলো ভাঙা ছিলো একদম খুবই বাজে এক্সপেরিন্স   
1472                                ভাল ঠান্ডা খা পাইছি   

                                   tokenized_clean_text  \
1780                    ['একটা', 'ভালো', 'না', 'খাবার']   
3102                ['বার্গার', 'ভালো', 'দামে', 'বেশি']   
1035                        ['পোলাও', 'মুটামু', 'ভালো']   
693                         ['মাংস', 'আরেকটু', 'দরকার']   
3887  ['বিরিয়ানির', 'স্বাদ', 'মোটামু', 'কাবাব', 'পু...   
3753  ['খা', 'ভালো', 'বাদাম', 'ভর্', 'দুইটা', 'ও', '...   
1560                                ['রোস্ট', 'এভারেজ']   
90    ['মশলার', 'সমৃদ্ধি', 'হালকা', 'মিষ্', 'মিষ্', ...   
3261  ['ফুচকা', 'সবগুলো', 'ভাঙা', 'ছিলো', 'একদম', 'খ...   
1472                   ['ভাল', 'ঠান্ডা', 'খা', 'পাইছি']   

                                    tokenized_stem_text  token_length  \
1780                               [এক, ভালো, না, খাবা]             4   
3102                        [বার্গার, ভালো, দামে, বেশি]             4   
1035                               [পোলা, মুটামু, ভালো]             3   
693                                 [মাংস, আরেকটু, দরক]             3   
3887  [বিরিয়ানির, স্বাদ, মোটামু, কাবাব, পুরানো, খেত...             8   
3753  [খা, ভালো, বাদাম, ভর্, দুই, ও, শুটকি, ভর্, এক,...            10   
1560                                    [রোস্ট, এভারেজ]             2   
90    [মশল, সমৃদ্ধি, হালকা, মিষ্, মিষ্, অনুভূতি, স্ব...             9   
3261  [ফুচকা, সব, ভাঙা, ছিলো, একদম, খুব, বাজে, এক্সপ...             8   
1472                             [ভাল, ঠান্ডা, খা, পাই]             4   

                                   postags_list  
1780                           [JQ, JJ, CX, NC]  
3102                           [NP, JJ, NC, JJ]  
1035                               [NC, NP, JJ]  
693                                [NC, NP, NC]  
3887           [NC, NC, NP, NC, JJ, VM, NC, CX]  
3753  [NP, JJ, NP, NP, JQ, CCD, NP, NP, JQ, VM]  
1560                                   [NP, NP]  
90         [NC, NC, NC, NP, NP, NC, NC, NP, JJ]  
3261          [NC, NC, NP, VM, ALC, JQ, NP, NP]  
1472                           [JJ, NP, NP, VM]

In [43]:
from collections import Counter
from functools import reduce

def find_no_class(count, class_name = ""):
    total = 0
    for key in count.keys():
      if key.startswith(class_name):
        total += count[key]        
    return total


def get_classes(row, grammatical_class = ""):
    count = Counter(row["postags_list"])
    try:
      return find_no_class(count, class_name = grammatical_class)/len(row["postags_list"])
    except ZeroDivisionError:
      return find_no_class(count, class_name = grammatical_class)
    

df["freqAdverbs"] = df.apply(lambda row: get_classes(row, "AMN"), axis = 1)
df["freqPreposition"] = df.apply(lambda row: get_classes(row, "PP"), axis = 1)
df["freqPronoun"] = df.apply(lambda row: get_classes(row, "PPR"), axis = 1)
df["freqVerbs"] = df.apply(lambda row: get_classes(row, "VM"), axis = 1)
df["freqAdjectives"] = df.apply(lambda row: get_classes(row, "JJ"), axis = 1)
df["freqNouns"] = df.apply(lambda row: get_classes(row, ("NC")), axis = 1)
df["freqEnglish"] = df.apply(lambda row: get_classes(row, ("RDF")), axis = 1)

In [44]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/w_post_tag_bangla_speech_tokenized.csv', index = False, header=True)

In [45]:
#parcentagewise null values
def missing_data_table(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data
missing_data_table(df)

Total   Percent
Speech                    1  0.000252
Label                     0  0.000000
text                      0  0.000000
clean_text                0  0.000000
tokenized_clean_text      0  0.000000
tokenized_stem_text       0  0.000000
token_length              0  0.000000
postags_list              0  0.000000
freqAdverbs               0  0.000000
freqPreposition           0  0.000000
freqPronoun               0  0.000000
freqVerbs                 0  0.000000
freqAdjectives            0  0.000000
freqNouns                 0  0.000000
freqEnglish               0  0.000000